# **Bike Sharing**
---
**<p>Stevens Institute of Technology</p>**
**<p>CS 559 WS - Machine Learning</p>**
##### Ravi Patel
##### 2/13/2020
---

## Objective
Build at least four regression models (e.g., linear, polynomial, non-linear) to predict the count of total rental bikes including both casual and registered. Explore data to reduce the number of features. Use K-fold cross validation and report the mean squared error (MSE) on the testing data. You need to write down every step in your experiment.

## 0. Setup

In [54]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import scipy
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
def calculate_metrics(X_train, y_train, X_test,y_test, model):
    model.fit(X_train, y_train)
    train_prediction = model.predict(X_train)
    mse_train_prediction = metrics.mean_squared_error(y_train,train_prediction)
    test_prediction = model.predict(X_test)
    mse_test_prediction = metrics.mean_squared_error(y_test,test_prediction)
    return mse_train_prediction, mse_test_prediction

## 1. Data Import

### 1.1 Importing of 'day.csv'

In [3]:
df_day = pd.read_csv("./data/Bike-Sharing-Dataset/day.csv")

In [4]:
df_day.shape

(731, 16)

In [5]:
df_day.columns


Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

### 1.2 Importing of 'hour.csv'

In [6]:
df_hour = pd.read_csv("./data/Bike-Sharing-Dataset/hour.csv")

In [7]:
df_hour.shape

(17379, 17)

In [8]:
df_hour.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

## 2. EDA and Preprocess

### 2.1 EDA of 'df_day'

#### 2.1.1 Describe 'df_day'

In [9]:
print('Dataset "df_day" contains {} rows/records and {} columns/features'.format(df_day.shape[0], df_day.shape[1]))

Dataset "df_day" contains 731 rows/records and 16 columns/features


In [10]:
df_day.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [11]:
df_day.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [12]:
# Notice that instant(column 0) is just a counter

In [13]:
df1_day = df_day.iloc[:,[2,3,4,5,6,7,8,9,10,11,12,13,14,15]]
df1_day.shape

(731, 14)

In [84]:
#Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df1_scaled_day = scaler.fit_transform(df1_day)

df1_scaled_day = pd.DataFrame(df1_scaled_day, columns=df1_day.columns)

df1_scaled_day.describe()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,0.498860,0.500684,0.501803,0.028728,0.499544,0.683995,0.197674,0.543594,0.518864,0.645649,0.346536,0.248291,0.525003,0.515687
std,0.370269,0.500342,0.313810,0.167155,0.334131,0.465233,0.272447,0.228091,0.213909,0.146457,0.159766,0.201474,0.225275,0.222873
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.333333,0.000000,0.272727,0.000000,0.166667,0.000000,0.000000,0.346343,0.339674,0.534704,0.232045,0.091989,0.357638,0.360101
50%,0.666667,1.000000,0.545455,0.000000,0.500000,1.000000,0.000000,0.547268,0.535113,0.644388,0.326928,0.208627,0.525845,0.520709
75%,0.666667,1.000000,0.818182,0.000000,0.833333,1.000000,0.500000,0.743002,0.695083,0.750857,0.434622,0.321009,0.686760,0.682697
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 2.2 EDA of 'df_hour'

#### 2.2.1 Describe 'df_hour'

In [75]:
df_hour.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [77]:
df_hour.describe()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [80]:
df_hour.shape
df1_hour = df_hour.iloc[:,2:]
df1_hour.shape

In [83]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df1_scaled_hour = scaler.fit_transform(df1_hour)

df1_scaled_hour = pd.DataFrame(df1_scaled_hour, columns=df1_hour.columns)

df1_scaled_hour.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,0.500547,0.502561,0.503434,0.502033,0.028770,0.500614,0.682721,0.141761,0.486722,0.475775,0.627229,0.223460,0.097210,0.173574,0.193097
std,0.368973,0.500008,0.312616,0.300626,0.167165,0.334295,0.465431,0.213119,0.196486,0.171850,0.192930,0.143811,0.134346,0.170832,0.185848
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.333333,0.000000,0.272727,0.260870,0.000000,0.166667,0.000000,0.000000,0.326531,0.333300,0.480000,0.122840,0.010899,0.038375,0.039959
50%,0.666667,1.000000,0.545455,0.521739,0.000000,0.500000,1.000000,0.000000,0.489796,0.484800,0.630000,0.228047,0.046322,0.129797,0.144467
75%,0.666667,1.000000,0.818182,0.782609,0.000000,0.833333,1.000000,0.333333,0.653061,0.621200,0.780000,0.298225,0.130790,0.248307,0.286885
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(17379, 15)

## 3. Regression of Day.csv

### 3.1 Data

In [15]:
X = df1_scaled_day.iloc[:,0:11]
y = df1_scaled_day.iloc[:,11:14]
X.head, y.head

(<bound method NDFrame.head of      season   yr  mnth  holiday   weekday  workingday  weathersit      temp  \
 0       0.0  0.0   0.0      0.0  1.000000         0.0         0.5  0.355170   
 1       0.0  0.0   0.0      0.0  0.000000         0.0         0.5  0.379232   
 2       0.0  0.0   0.0      0.0  0.166667         1.0         0.0  0.171000   
 3       0.0  0.0   0.0      0.0  0.333333         1.0         0.0  0.175530   
 4       0.0  0.0   0.0      0.0  0.500000         1.0         0.0  0.209120   
 ..      ...  ...   ...      ...       ...         ...         ...       ...   
 726     0.0  1.0   1.0      0.0  0.666667         1.0         0.5  0.243025   
 727     0.0  1.0   1.0      0.0  0.833333         1.0         0.5  0.241986   
 728     0.0  1.0   1.0      0.0  1.000000         0.0         0.5  0.241986   
 729     0.0  1.0   1.0      0.0  0.000000         0.0         0.0  0.245101   
 730     0.0  1.0   1.0      0.0  0.166667         1.0         0.5  0.195259   
 
        

### 3.2 Linear Regression 

#### 3.2.1 Without K-Fold

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [17]:
X_train.shape, y_train.shape

((584, 11), (584, 3))

In [18]:
X_test.shape, y_test.shape

((147, 11), (147, 3))

In [19]:
lr = LinearRegression()
train_error, validation_error = calculate_metrics(X_train, y_train, X_test, y_test, lr)

In [20]:
print('Train Errors: {}'.format(train_error))

Train Errors: 0.010791058291067442


In [21]:
print('Validation Errors: {}'.format(validation_error))

Validation Errors: 0.010174428930630577


In [22]:
lr.coef_

array([[ 0.04794878,  0.08348112, -0.04031611, -0.08681917,  0.03564254,
        -0.23684889, -0.0751243 ,  0.34810156,  0.11202772, -0.0920512 ,
        -0.12040152],
       [ 0.19637273,  0.25156691, -0.04394499, -0.04038236,  0.03946158,
         0.14140992, -0.1480981 ,  0.11395032,  0.26146225, -0.07287476,
        -0.11241407],
       [ 0.17527462,  0.23318639, -0.05082378, -0.06621812,  0.04541886,
         0.0198141 , -0.14746331,  0.22728372,  0.25226392, -0.09416027,
        -0.1367819 ]])

#### 3.2.2 With K-Fold

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [24]:
# def calculate_metrics(X_train, y_train, X_test,y_test, model):
#     model.fit(X_train, y_train)
#     train_prediction = model.predict(X_train)
#     mse_train_prediction = metrics.mean_squared_error(y_train,train_prediction)
#     test_prediction = model.predict(X_test)
#     mse_test_prediction = metrics.mean_squared_error(y_test,test_prediction)
#     return mse_train_prediction, mse_test_prediction

In [25]:
K = 10
folds = list(KFold(n_splits=K, shuffle=True).split(X,y))

lr = LinearRegression()
train_errors = []
validation_errors = []

for k, (train_index, valid_index) in enumerate(folds):
    X_train, X_valid = X.iloc[train_index,], X.iloc[valid_index,]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
    train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, lr)
    
    train_errors.append(train_error)
    validation_errors.append(validation_error)


In [26]:
print('Train Errors: {}'.format(train_errors))

Train Errors: [0.010711084271310053, 0.010464513479098428, 0.010812349572368056, 0.010235225197874773, 0.010388699872020459, 0.010529706066633324, 0.01074131437888542, 0.010996208377985652, 0.010645692285690551, 0.01039504347982457]


In [27]:
print('Validation Errors: {}'.format(validation_errors))

Validation Errors: [0.009952012393635159, 0.01239856276349395, 0.009039854332105109, 0.014305926168502394, 0.01293048525040274, 0.011900338929545506, 0.009661591602088291, 0.0073247951762190336, 0.01169200560932711, 0.012756171138737192]


### 3.3 Lasso Regression

#### 3.3.1 Without K-Fold

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [29]:
X_train.shape, y_train.shape

((584, 11), (584, 3))

In [30]:
X_test.shape, y_test.shape

((147, 11), (147, 3))

In [31]:
lassr = Lasso()
train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, lassr)

In [32]:
print('Train Errors: {}'.format(train_error))

Train Errors: 0.04651997948725651


In [33]:
print('Validation Errors: {}'.format(validation_error))

Validation Errors: 0.04303994822606697


#### 3.3.2 With K-Fold

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [35]:
X_train.shape, y_train.shape

((584, 11), (584, 3))

In [36]:
X_test.shape, y_test.shape

((147, 11), (147, 3))

In [37]:
K = 10
folds = list(KFold(n_splits=K, shuffle=True).split(X,y))

lassr = Lasso()
train_errors = []
validation_errors = []

for k, (train_index, valid_index) in enumerate(folds):
    X_train, X_valid = X.iloc[train_index,], X.iloc[valid_index,]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
    train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, lr)
    
    train_errors.append(train_error)
    validation_errors.append(validation_error)

In [38]:
print('Train Errors: {}'.format(train_errors))

Train Errors: [0.010699874896022707, 0.010821149157779994, 0.010656805542610752, 0.010632650550625714, 0.010812877851159608, 0.010847919027107434, 0.010480658862977379, 0.010435207256367374, 0.01018106222765497, 0.010323746252545826]


In [39]:
print('Validation Errors: {}'.format(validation_errors))

Validation Errors: [0.010046701448185778, 0.00896648433361813, 0.011972716175776776, 0.010678436094496301, 0.008993663296901102, 0.008853513066354384, 0.012126388906137332, 0.012641877278794143, 0.014890715114660119, 0.013585991499190554]


### 3.4 Ridge Regression

#### 3.4.1 Without K-Fold

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [58]:
ridge = Ridge()
params = {'alpha':[1e-12, 1e-8, 1e-4, 1e-2, 1, 5 ]}
rr = GridSearchCV(ridge, params, scoring='neg_mean_squared_error', cv=5)
train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, rr)

In [59]:
print(rr.best_params_)

{'alpha': 5}


In [60]:
print(rr.best_score_)

-0.013441411189984398


In [63]:
print(train_error)

0.010466832717836471


In [62]:
print(validation_error)

0.013654895592494104


#### 3.4.2 With K-Fold

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [66]:
K = 10
folds = list(KFold(n_splits=K, shuffle=True).split(X,y))

ridge = Ridge()
params = {'alpha':[1e-12, 1e-8, 1e-4, 1e-2, 1, 5 ]}
rr = GridSearchCV(ridge, params, scoring='neg_mean_squared_error', cv=5)
train_errors = []
validation_errors = []

for k, (train_index, valid_index) in enumerate(folds):
    X_train, X_valid = X.iloc[train_index,], X.iloc[valid_index,]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
    train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, rr)
    
    train_errors.append(train_error)
    validation_errors.append(validation_error)

In [67]:
print(rr.best_params_)

{'alpha': 5}


In [68]:
print(rr.best_score_)

-0.013885301564359995


In [71]:
print(train_errors)

[0.010579235961955003, 0.010870022337076587, 0.010386395018852098, 0.01042457732124974, 0.010590627352326557, 0.01072747391873279, 0.011091608231540791, 0.0108143429046007, 0.011039702201835767, 0.01083499808543393]


In [72]:
print(validation_errors)

[0.012558051393284428, 0.009674712722882907, 0.013801513191036742, 0.01404619107762991, 0.012516558795077461, 0.01108729277548245, 0.007927316100394648, 0.010231347256490537, 0.008196130854374019, 0.010827371072848346]


## 4. Regression of Hour.csv

### 4.1 Data

In [85]:
X = df1_scaled_hour.iloc[:,0:12]
y = df1_scaled_hour.iloc[:,12:15]
X.head, y.head

### 4.2 Linear Regression

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [87]:
X_train.shape, y_train.shape

((13903, 12), (13903, 3))

In [88]:
X_test.shape, y_test.shape

((3476, 12), (3476, 3))

In [91]:
K = 10
folds = list(KFold(n_splits=K, shuffle=True).split(X,y))

lr = LinearRegression()
train_errors = []
validation_errors = []

for k, (train_index, valid_index) in enumerate(folds):
    X_train, X_valid = X.iloc[train_index,], X.iloc[valid_index,]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
    train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, lr)
    
    train_errors.append(train_error)
    validation_errors.append(validation_error)


In [92]:
print('Train Errors: {}'.format(train_errors))

Train Errors: [0.016745097176073327, 0.016744489793182545, 0.01681680499297356, 0.01680828995474139, 0.016839825640276456, 0.016737337410563973, 0.016800949524336087, 0.01671905090962204, 0.016718568174736766, 0.01678977420328907]


In [93]:
print('Validation Errors: {}'.format(validation_errors))

Validation Errors: [0.017035359175272772, 0.017039753561213938, 0.016400734567930016, 0.01646841651761595, 0.016180997260287877, 0.017105738070285103, 0.016531179678019783, 0.017271024216452376, 0.017275096113098475, 0.016634828385037524]


### 4.3 Lasso Regression

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [95]:
K = 10
folds = list(KFold(n_splits=K, shuffle=True).split(X,y))

lassr = Lasso()
train_errors = []
validation_errors = []

for k, (train_index, valid_index) in enumerate(folds):
    X_train, X_valid = X.iloc[train_index,], X.iloc[valid_index,]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
    train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, lr)
    
    train_errors.append(train_error)
    validation_errors.append(validation_error)

In [96]:
print('Train Errors: {}'.format(train_errors))

Train Errors: [0.0169244065738082, 0.016776789333527633, 0.016744645318736057, 0.016829127151410713, 0.016717105483917843, 0.016769049004123984, 0.016771377894088987, 0.01657385310248763, 0.016698905098630045, 0.016912590756731318]


In [97]:
print('Validation Errors: {}'.format(validation_errors))

Validation Errors: [0.015432270101090962, 0.0167598966326604, 0.017035283203824997, 0.01628311851202992, 0.01728898256289584, 0.016817553778687776, 0.016807890409023477, 0.018579763792352103, 0.0174528096792383, 0.01552895872922233]


### 4.4 Ridge Regression

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [99]:
K = 10
folds = list(KFold(n_splits=K, shuffle=True).split(X,y))

ridge = Ridge()
params = {'alpha':[1e-12, 1e-8, 1e-4, 1e-2, 1, 5 ]}
rr = GridSearchCV(ridge, params, scoring='neg_mean_squared_error', cv=5)
train_errors = []
validation_errors = []

for k, (train_index, valid_index) in enumerate(folds):
    X_train, X_valid = X.iloc[train_index,], X.iloc[valid_index,]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    
    train_error, validation_error = calculate_metrics(X_train, y_train, X_valid, y_valid, rr)
    
    train_errors.append(train_error)
    validation_errors.append(validation_error)

In [100]:
print('Train Errors: {}'.format(train_errors))

Train Errors: [0.01675832010158598, 0.016895810515385124, 0.016768350276581703, 0.016665909997238052, 0.016819119441021182, 0.016778758153964218, 0.016809061111980893, 0.016719209600245533, 0.01668196511098867, 0.016835849285535835]


In [101]:
print('Validation Errors: {}'.format(validation_errors))

Validation Errors: [0.016924864040429238, 0.015685439835686276, 0.016836972686429547, 0.017765685820616692, 0.01637693159687447, 0.016742688857094623, 0.016476389086326194, 0.01727336533794233, 0.017610942174888056, 0.016232347575627266]
